In [1]:
%pwd

'd:\\Projects\\Alzeimer\\Alzheimer-s-Disease-Detection\\research'

In [3]:
import os
os.chdir("../")
%pwd

'd:\\Projects\\Alzeimer\\Alzheimer-s-Disease-Detection'

In [5]:
from dataclasses import dataclass

@dataclass
class ModelTrainerArtifcats:
    model_dir : str 



In [6]:
from alzheimer_disease.entity.config_entity import training_pipeline_config, TrainingPipelineConfig,DataIngestionConfig
from alzheimer_disease.constant.training_pipeline import *




@dataclass
class ModelTrainerConfig:
    model_trainer_dir: str = os.path.join(
        training_pipeline_config.artifacts_dir, MODEL_TRAINER_DIR_NAME)

    model_save: str = os.path.join(model_trainer_dir, MODEL_NAME)
    batch_size : int = MODEL_TRAINER_BATCH_SIZE
    epochs : int = MODEL_TRAINER_EPOCHS
    checkpoint_dir: str = os.path.join(model_trainer_dir, MODEL_CHECKPOINT_DIR_NAME)
    image_size: int = IMAGE_SIZE
    channels: int = CHANNELS
    input_shape: tuple = INPUT_SHAPE
    classes: int = CLASSES
    validation_split: int = VALIDATION_SPLIT
    optimizer: str = OPTIMIZER
    loss: str = LOSS
    metrics: str = METRICS
    patience: int = PATIENCE

    

  


In [ ]:
from alzheimer_disease.entity.artifacts_entity import DataIngestionArtifact
from alzheimer_disease.utils.main_utils import create_callbacks, create_datasets
from alzheimer_disease.exception import AlzException
from alzheimer_disease.logger import logging
import tensorflow as tf
import sys


class ModelTrainer:

    def __init__(self, data_ingestion_artifact: DataIngestionArtifact, model_trainer_config: ModelTrainerConfig) -> ModelTrainerArtifcats:
        logging.info(f"{'>>'*20}Model Training{'<<'*20}")
        self.data_ingestion_artifact = data_ingestion_artifact
        self.model_trainer_config = model_trainer_config
    
    def build_model(self, input_shape):
        """
        Build and compile the convolutional neural network model.

        Args:
            input_shape (tuple): Shape of the input images in the format (height, width, channels).

        Returns:
            model (tf.keras.Model): Compiled convolutional neural network model.
        """
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
            tf.keras.layers.MaxPool2D((2, 2)),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPool2D((2, 2)),
            tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPool2D((2, 2)),
            tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPool2D((2, 2)),
            tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPool2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(self.model_trainer_config.classes, activation='softmax')
        ])

        return model
        
    def initiate_model_training (self):
        logging.info("Preparing TensorFlow Dataset for training")
        train, val, test, class_name = create_datasets(self.data_ingestion_artifact.train_path, 
                                                       self.data_ingestion_artifact.test_path,
                                                       image_size=self.model_trainer_config.image_size, 
                                                       batch_size=self.model_trainer_config.batch_size,
                                                       validation_split=self.model_trainer_config.validation_split)
        
        logging.info(f"TensorFlow Dataset prepared and has {class_name}")
        
        model = self.build_model((self.model_trainer_config.image_size,
                                  self.model_trainer_config.image_size,
                                  self.model_trainer_config.channels))
        
        logging.info("Convolutional Neural Network Model Built")
        
        model.compile(optimizer=self.model_trainer_config.optimizer,
                      loss=self.model_trainer_config.loss,
                      metrics=self.model_trainer_config.metrics)
        
        logging.info("Convolutional Neural Network Model Compiled")
        
        callbacks = create_callbacks(self.model_trainer_config.checkpoint_dir, patience=self.model_trainer_config.patience)
        
        logging.info("Callbacks created")
        
        history = model.fit(train,
                            validation_data=val,
                            epochs=self.model_trainer_config.epochs,
                            callbacks=callbacks)
        
        logging.info("Training Completed")
        
        model.save(self.model_trainer_config.model_save)
        
        logging.info("Model Saved")
        
        model_trainer_artifact = ModelTrainerArtifcats(model=self.model_trainer_config.model_save)
        
        logging.info(f"Model Trained Artifact Created: {model_trainer_artifact}")
        
        return model_trainer_artifact
